# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
More detail in the writeup report. Rubric: [link](https://review.udacity.com/#!/rubrics/322/view)

## Import Packages

In [15]:
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

## Video Input and Output

In [16]:
#ToDo Replace the video input and out

video_output = 'test_videos_output/solidWhiteRight.mp4'
video_input = 'test_videos/solidWhiteRight.mp4'

#video_output = 'test_videos_output/solidYellowLeft.mp4'
#video_input = 'test_videos/solidYellowLeft.mp4'

## Helper Functions

In [20]:
def canny(img, low_threshold, high_threshold):
    """Applies (Gauss smoothing and) the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def region_of_interest(img, vertices):
    """
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, yCutOff, color=[255, 0, 0], thickness=2):
    """
    Line segments separated by their slope: ((y2-y1)/(x2-x1))
    Lines with below average slope: Right lane; Others: Left lane
    Lines representing these lanes are drawn on the image
    inplace (mutates the image) in the region of interest.
    yCutOff variable is from RoI code segment.
    """
    
    # Calculating slope for line segments
    slope = np.zeros(len(lines))
    i = 0
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope[i] = ((y2-y1)/(x2-x1))
        i += 1
    avgslope = np.average(slope)
    
    # Calculating the left and right lane line parameters
    # y=m*x+b
    leftlanem = 0
    leftlaneb = 0
    rightlanem = 0
    rightlaneb = 0
    rl = 0
    ll = 0
    for line in lines:
        for x1,y1,x2,y2 in line:
            if ((y2-y1)/(x2-x1)) > avgslope:
                rightlanem += (y2-y1)/(x2-x1)
                rightlaneb += (x2*y1-x1*y2)/(x2-x1)
                rl += 1
            else:
                leftlanem += (y2-y1)/(x2-x1)
                leftlaneb += (x2*y1-x1*y2)/(x2-x1)
                ll += 1
    leftlanem /= ll
    leftlaneb /= ll
    rightlanem /= rl
    rightlaneb /= rl
    
    # Calculating the intersection of the RoI and the lanes
    leftx1 = int((yCutOff-leftlaneb)/leftlanem)
    leftx2 = int((img.shape[0]-leftlaneb)/leftlanem)
    rightx1 = int((yCutOff-rightlaneb)/rightlanem)
    rightx2 = int((img.shape[0]-rightlaneb)/rightlanem)
    
    # Drawing the lanes on the image
    cv2.line(img, (leftx1, int(yCutOff)), (leftx2, img.shape[0]), color, thickness)
    cv2.line(img, (rightx1, int(yCutOff)), (rightx2, img.shape[0]), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap, yCutOff):
    """
    `img` should be the output of a Canny transform.    
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines, yCutOff)
    return line_img

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    `initial_img` should be the image before any processing.
    The result image is computed as follows:
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

def process_image(image):
    # Image processing pipeline
    # Returns original image with detected left and right lane

    # Converting dominantly white and yellow pixels to white 255,255,255
    # white 255,255,255; light yellow 255,255,150; dark yellow 200,200,0
    imgColorSelect = np.copy(image)
    red_threshold = 200
    green_threshold = 200
    blue_threshold = 0
    thresholds = (imgColorSelect[:,:,0] < red_threshold) \
               | (imgColorSelect[:,:,1] < green_threshold) \
               | (imgColorSelect[:,:,2] < blue_threshold)
    imgColorSelect[thresholds] = [0,0,0]
    imgBinary = np.copy(imgColorSelect)
    thresholds2 = (imgBinary[:,:,0] > 1) # non black pixels
    imgBinary[thresholds2] = [255,255,255]

    # Gaussian smoothing -> not implemented now (CV Canny applies Gaussian smoothing internally) 
    # Canny edge detection (threshold recommendation: low to high = 1:2 or 1:3)
    low_threshold = 60
    high_threshold = 180
    imgCanny = canny(imgBinary, low_threshold, high_threshold)

    # Masking to get Region of Interest 
    ysize = imgCanny.shape[0]
    xsize = imgCanny.shape[1]
    lineWidth = 800  # Close to car
    xCamOffset1 = 20 # Camera is off center in case of test_images, close to car
    xCamOffset2 = 10 # Camera is off center in case of test_images, far from car
    yCutOff = ysize/2 + 50 # Cuting off too distant lines (somewhere over ~50 m)
    vertices = np.array([[(xsize/2 - lineWidth/2 + xCamOffset1,ysize),
                          (xsize/2 - 120 + xCamOffset2, yCutOff),
                          (xsize/2 + 120 + xCamOffset2, yCutOff),
                          (xsize/2 + lineWidth/2 + xCamOffset1,ysize)]], dtype=np.int32)
    imgRoI = region_of_interest(imgCanny, vertices)

    # Hough transform
    rho = 3            # distance resolution in pixels of the Hough grid, 3
    theta = np.pi/180  # angular resolution in radians of the Hough grid, pi/180
    threshold = 25     # minimum number of votes (intersections in Hough grid cell), 15
    min_line_len = 40  # minimum number of pixels making up a line, 40
    max_line_gap = 60  # maximum gap in pixels between connectable line segments, 20
    imgHugh = hough_lines(imgRoI, rho, theta, threshold, min_line_len, max_line_gap, yCutOff)

    # Overlay original image with detected line segments
    result = weighted_img(imgHugh, image, α=0.8, β=1., γ=0.)

    return result

## Test on Videos

In [21]:
clip1 = VideoFileClip(video_input)
video_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time video_clip.write_videofile(video_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:19<00:00, 11.18it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

CPU times: user 8.65 s, sys: 269 ms, total: 8.92 s
Wall time: 21.7 s


In [22]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output))